In [1]:
import pandas as pd
import ORE as ql

In [2]:
today = ql.Date(27, ql.April, 2025)
ql.Settings.instance().evaluationDate = today

In [3]:
sample_rates = pd.DataFrame(
    [
        (ql.Date(27, 10, 2021), 2.0229, 1.4510, 1.5131, 2.6893),
        (ql.Date(27, 1, 2022), 2.0645, 1.4416, 1.4943, 2.6919),
        (ql.Date(27, 4, 2022), 2.0414, 1.4520, 1.4764, 2.7306),
        (ql.Date(27, 10, 2022), 2.1630, 1.4344, 1.4969, 2.8107),
        (ql.Date(27, 10, 2023), 2.4638, 1.5635, 1.6532, 3.1354),
        (ql.Date(27, 10, 2024), 2.7187, 1.6500, 1.7510, 3.3850),
        (ql.Date(27, 10, 2025), 2.9055, 1.6959, 1.8410, 3.5358),
        (ql.Date(27, 10, 2026), 3.0673, 1.7660, 1.9268, 3.6438),
        (ql.Date(27, 10, 2027), 3.1615, 1.8310, 1.9669, 3.7127),
        (ql.Date(27, 10, 2028), 3.2325, 1.8959, 2.0346, 3.7374),
        (ql.Date(27, 10, 2029), 3.3049, 1.9930, 2.1263, 3.7842),
        (ql.Date(27, 10, 2030), 3.3584, 2.0272, 2.1832, 3.7844),
        (ql.Date(27, 10, 2031), 3.4023, 2.0744, 2.2599, 3.7927),
        (ql.Date(27, 10, 2036), 3.5657, 2.3011, 2.4406, 3.8902),
        (ql.Date(27, 10, 2041), 3.6191, 2.3882, 2.5331, 3.9043),
        (ql.Date(27, 10, 2046), 3.6199, 2.3762, 2.5225, 3.8677),
        (ql.Date(27, 10, 2051), 3.6208, 2.3401, 2.4926, 3.8204),
    ],
    columns=[
        "date",
        "SOFR",
        "ESTR",
        "Euribor3M",
        "CORRA",
    ],
)

In [4]:
sample_rates

,date,SOFR,ESTR,Euribor3M,CORRA
0,"October 27th, 2021",2.0229,1.4510,1.5131,2.6893
1,"January 27th, 2022",2.0645,1.4416,1.4943,2.6919
2,"April 27th, 2022",2.0414,1.4520,1.4764,2.7306
3,"October 27th, 2022",2.1630,1.4344,1.4969,2.8107
4,"October 27th, 2023",2.4638,1.5635,1.6532,3.1354
5,"October 27th, 2024",2.7187,1.6500,1.7510,3.3850
6,"October 27th, 2025",2.9055,1.6959,1.8410,3.5358
7,"October 27th, 2026",3.0673,1.7660,1.9268,3.6438
8,"October 27th, 2027",3.1615,1.8310,1.9669,3.7127
9,"October 27th, 2028",3.2325,1.8959,2.0346,3.7374


In [5]:
def sample_curve(tag):
    curve = ql.ZeroCurve(
        sample_rates["date"], sample_rates[tag] / 100, ql.Actual365Fixed()
    )
    return ql.YieldTermStructureHandle(curve)

In [6]:
sofr_curve = sample_curve("SOFR")
estr_curve = sample_curve("ESTR")
euribor_curve = sample_curve("Euribor3M")
corra_curve = sample_curve("CORRA")

In [7]:
sofr = ql.Sofr(sofr_curve)
euribor = ql.Euribor3M(euribor_curve)

In [8]:
basis_quotes = [
    (ql.Period(1, ql.Years), -14.5),
    (ql.Period(18, ql.Months), -18.5),
    (ql.Period(2, ql.Years), -20.5),
    (ql.Period(3, ql.Years), -23.75),
    (ql.Period(4, ql.Years), -25.5),
    (ql.Period(5, ql.Years), -26.5),
    (ql.Period(7, ql.Years), -26.75),
    (ql.Period(10, ql.Years), -26.25),
    (ql.Period(15, ql.Years), -24.75),
    (ql.Period(20, ql.Years), -23.25),
    (ql.Period(25, ql.Years), -20.50),
]

In [9]:
fixing_days = 2
calendar = ql.TARGET()
convention = ql.Following
end_of_month = True

In [24]:
base_index = sofr
other_index = euribor
collateral_curve = sofr_curve
base_is_collateral = True
basis_on_base = False
base_resets = False
frequency = ql.Quarterly

helpers = [
    ql.MtMCrossCurrencyBasisSwapRateHelper(
        ql.makeQuoteHandle(basis / 10000),
        tenor,
        fixing_days,
        calendar,
        convention,
        end_of_month,
        base_index,
        other_index,
        collateral_curve,
        base_is_collateral,
        basis_on_base,
        base_resets,
        frequency,
    )
    for tenor, basis in basis_quotes
]

In [25]:
discount_curve = ql.PiecewiseLinearZero(
    today, helpers, ql.Actual365Fixed()
)

In [26]:
pd.DataFrame(
    discount_curve.nodes(), columns=["date", "rate"]
).style.format({"rate": "{:.4%}"})

,date,rate
0,"April 27th, 2025",2.0462%
1,"April 30th, 2026",2.0462%
2,"October 30th, 2026",2.0446%
3,"April 30th, 2027",2.0033%
4,"May 2nd, 2028",1.9990%
5,"April 30th, 2029",2.0702%
6,"April 30th, 2030",2.1363%
7,"April 30th, 2032",2.2478%
8,"April 30th, 2035",2.3280%
9,"April 30th, 2040",2.4262%


## A slightly more complex case


In [20]:
euribor = ql.Euribor3M(euribor_curve)
corra = ql.Corra(corra_curve)

In [21]:
base_index = euribor
other_index = corra
collateral_curve = estr_curve
base_is_collateral = True
basis_on_base = False
base_resets = False
frequency = ql.Quarterly

helpers = [
    ql.MtMCrossCurrencyBasisSwapRateHelper(
        ql.makeQuoteHandle(basis / 10000),
        tenor,
        fixing_days,
        calendar,
        convention,
        end_of_month,
        base_index,
        other_index,
        collateral_curve,
        base_is_collateral,
        basis_on_base,
        base_resets,
        frequency,
    )
    for tenor, basis in basis_quotes
]

In [22]:
discount_curve = ql.PiecewiseLinearZero(
    today, helpers, ql.Actual365Fixed()
)

In [23]:
pd.DataFrame(
    discount_curve.nodes(), columns=["date", "rate"]
).style.format({"rate": "{:.4%}"})

,date,rate
0,"April 27th, 2025",3.6426%
1,"April 30th, 2026",3.6426%
2,"October 30th, 2026",3.6407%
3,"April 30th, 2027",3.6645%
4,"May 2nd, 2028",3.6399%
5,"April 30th, 2029",3.6201%
6,"April 30th, 2030",3.5851%
7,"April 30th, 2032",3.4957%
8,"April 30th, 2035",3.5733%
9,"April 30th, 2040",3.6095%
